In [2]:
!pip install findspark

In [3]:
import findspark

In [4]:
findspark.init()

In [5]:
import pyspark

In [6]:
from pyspark.sql import SparkSession 

In [31]:
from pyspark.sql.functions import date_format,col, countDistinct, count_distinct

In [7]:
spark = SparkSession.builder.appName('finderApp').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/19 12:32:36 WARN Utils: Your hostname, de-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/07/19 12:32:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/19 12:32:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark

In [13]:
data = {
(1,1,'2019-07-20','open_session'),
(1,1,'2019-07-20','scroll_down'),
(1,1,'2019-07-20','end_session'),
(2,4,'2019-07-20','open_session'),
(2,4,'2019-07-21','send_message'),
(2,4,'2019-07-21','end_session'),
(3,2,'2019-07-21','open_session'),
(3,2,'2019-07-21','send_message'),
(3,2,'2019-07-21','end_session'),
(4,3,'2019-06-25','open_session'),
(4,3,'2019-06-25','end_session')
}

schema = ["user_id","session_id","activity_date","activity_type"]
df_raw = spark.createDataFrame(data = data,schema = schema)
df1 = df_raw.select("user_id","session_id",date_format("activity_date",'yyyy-MM-dd').alias("activity_date"), "activity_type")

In [16]:
df1.show(truncate=False)

+-------+----------+-------------+-------------+
|user_id|session_id|activity_date|activity_type|
+-------+----------+-------------+-------------+
|1      |1         |2019-07-20   |end_session  |
|1      |1         |2019-07-20   |scroll_down  |
|3      |2         |2019-07-21   |open_session |
|4      |3         |2019-06-25   |end_session  |
|2      |4         |2019-07-20   |open_session |
|3      |2         |2019-07-21   |end_session  |
|2      |4         |2019-07-21   |send_message |
|2      |4         |2019-07-21   |end_session  |
|3      |2         |2019-07-21   |send_message |
|1      |1         |2019-07-20   |open_session |
|4      |3         |2019-06-25   |open_session |
+-------+----------+-------------+-------------+



In [29]:
df1.where( (col("activity_date") >= '2019-06-28') & (col("activity_date") <= '2019-07-27'))\
    .select(col("activity_date"), col("user_id"))\
    .groupby(col("activity_date")).agg(countDistinct("user_id")).alias("distinct_user_count").show(truncate=False)

IndentationError: unexpected indent (1889680727.py, line 3)

In [38]:
df1.where( (col("activity_date") >= '2019-06-28') & (col("activity_date") <= '2019-07-27'))\
    .groupby(col("activity_date"))\
    .agg(countDistinct(col("user_id"))).alias("distinct_user_count").show(truncate=False)

+-------------+-----------------------+
|activity_date|count(DISTINCT user_id)|
+-------------+-----------------------+
|2019-07-21   |2                      |
|2019-07-20   |2                      |
+-------------+-----------------------+



## SQL Solution

<pre>
SELECT activity_date, count(DISTINCT user_id )
FROM Activity
WHERE activity_date BETWEEN ADDDATE('2019-07-27' INTERVAL -29 DAY) AND '2019-07-27'
GROUP BY activity_date;
</pre>